In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import CRUD_Python_Module

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "aacpassword"

# Connect to database via CRUD Module
shelter = CRUD_Python_Module.AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
html.Img(
    src='data:image/png;base64,{}'.format(encoded_image.decode()),
    style={'width': '150px', 'height': 'auto'}
),
    html.H1("Module 5 Asssignment - Alex"),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='buttonRow', 
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
                    html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                    html.Button(id='submit-button-four', n_clicks=0, children='Reset'),
                ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,                # prevent accidental edits
                         filter_action="native",        # allow filtering
                         sort_action="native",          # allow sorting
                         sort_mode="multi",             # allow multi-column sorting
                         row_selectable="single",       # only one row at a time
                         selected_rows=[],              # initialize empty selection
                         page_action="native",          # enable pagination
                         page_current=0,
                         page_size=10,                  # show 10 rows per page
                         style_table={'overflowX': 'auto'},
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '100%', 'height': '500px'}
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '75%', 'height': '500px'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    

    
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
    [Input('submit-button-one','n_clicks'),
     Input('submit-button-two','n_clicks'),
     Input('submit-button-three','n_clicks'),
     Input('submit-button-four','n_clicks')]
)
def update_dashboard(btn1, btn2, btn3, btn4):
    ctx = dash.callback_context # looked this up after trying to do on_click for 4 separate buttons and getting 4 errors...
    if not ctx.triggered:
        dff = df
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'submit-button-one':
            filter_type = "Water Rescue"
        elif button_id == 'submit-button-two':
            filter_type = "Mountain Rescue"
        elif button_id == 'submit-button-three':
            filter_type = "Disaster Rescue"
        else:
            filter_type = "Reset"

        # --- Apply MongoDB filtering logic here ---
        if filter_type == "Reset":
            dff = df #Change the view to the default one
            
        elif filter_type == "Water Rescue":
            breeds = ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]
            dff = df[(df['breed'].isin(breeds)) &
                     (df['sex_upon_outcome'] == "Intact Female") &
                     (df['age_upon_outcome_in_weeks'].between(26,156))] #otherwise filter it to the specific needs of 
            
        elif filter_type == "Mountain Rescue":
            breeds = ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]
            dff = df[(df['breed'].isin(breeds)) &
                     (df['sex_upon_outcome'] == "Intact Male") &
                     (df['age_upon_outcome_in_weeks'].between(26,156))]
            
        elif filter_type == "Disaster Rescue":
            breeds = ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]
            dff = df[(df['breed'].isin(breeds)) &
                     (df['sex_upon_outcome'] == "Intact Male") &
                     (df['age_upon_outcome_in_weeks'].between(20,300))]

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    data=dff.to_dict('records')
    return data, columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if dff.empty:
        return []

    # Count occurrences of each breed
    counts = dff['breed'].value_counts().reset_index()
    counts.columns = ['breed', 'count']

    # Compute percentages
    total = counts['count'].sum()
    counts['pct'] = counts['count'] / total

    # Filter to only breeds above 1%
    filtered = counts[counts['pct'] >= 0.01].copy()

    # Optional: group the rest into "Other"
    if len(filtered) < len(counts):
        other_count = counts[counts['pct'] < 0.01]['count'].sum()
        filtered.loc[len(filtered)] = ['Other', other_count, other_count / total]

    fig = px.pie(filtered, names='breed', values='count', title='Preferred Animals')

    return [dcc.Graph(figure=fig)]


    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if index is None or len(index) == 0:
        return []
    
    # Get selected row
    dff = pd.DataFrame(viewData)
    row = dff.iloc[index[0]]
    
    # Extract lat/long
    lat = row['location_lat']
    lon = row['location_long']
    
    # Return leaflet map
    return [
    dl.Map(center=[lat, lon], zoom=12, style={'width': '100%', 'height': '500px'}, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(row['animal_type']),
            dl.Popup([
                html.H3(f"Animal ID: {row['animal_id']}"),
                html.P(f"Breed: {row['breed']}"),
                html.P(f"Age: {row['age_upon_outcome']}")
            ])
        ])
    ])
]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://portalvisitor-visionnever-3000.codio.io/proxy/8050/
